## Open up the scrubbed data file
This notebeook opens up the file named `responses_scrubbed_tagged.csv` and asks the user to tag each respondent's free text responses in order to be able to later quantify how many respondent's mentioned certain common complaints or suggestions.

If starting this tagging process from scratch for the first time, make sure to run the `1.cleanup.ipnyb` script first.  Then make a copy of the `responses_scrubbed.csv` created by that script and name it `responses_scrubbed_tagged.csv` prior to running this notebook.

If you have already tagged responses previously, make a backup of the file named `responses_scrubbed_tagged.csv`.  Run the `1.cleanup.ipynb` notebook, then run this notebook and make sure to set the `start_from` variable to the appropriate new row number in the code.  Then use a spreadsheet program to copy the tags in the `problem_tags` and `suggestion_tags` from your backup file into the newly-generated `responses_scrubbed_tagged.csv` file.


In [1]:
import numpy as np
import pandas as pd
from IPython.display import display, Markdown, Latex, clear_output

# open the file... this should be a responses_scrubbed.csv generated by the cleanup script to start with
df = pd.read_csv('responses_scrubbed_tagged.csv', index_col=None, parse_dates=['date'])

#don't truncate results
pd.set_option('display.max_rows', None)

## Which row to start tagging?
If this is the first time running this notebook, set it to zero.  If you have previously tagged rows, set this to the first row number you have not yet tagged.

In [2]:
# what row index number to start from (useful if interrputed)
start_from = 560 #default to zero if starting from beginning


## Categorize every free text response from every respondent

In [3]:
# fields containing free text responses
fields = ['feelings', 'problem_areas', 'suggested_improvements', 'additional_comments', 'business_additional_comments', 'final_comments']

def get_unique_values_from_csv_field(column, df):
    combo_vals = df[column].tolist() # contains comma-separated sets of tags
    single_vals = [] # will contain single tags
    for combo in combo_vals:
        list_of_vals = str(combo).split(',') # break up comma-seprated values into single values
        list_of_vals = [val.strip() for val in list_of_vals] # remove whitespace
        single_vals.extend(list_of_vals) # add single values to the ultimate list
        
    # return the value counts
    return single_vals
    
# start with a blank list of tags
problem_tags = []
suggestion_tags = []

# count rows we loop through
counter = 0

# loop through every row in the data
for index, row in df.iterrows():
    
    #skip to starting row
    if counter < start_from:
        counter += 1
        continue

    # determine whether we have already tagged this respondent's free text responses
    my_problems = ''
    my_suggestions = ''
    try:
        my_problems = df.loc[index]['problem_tags']
        my_suggestions = df.loc[index]['suggestion_tags']
        problem_tags = get_unique_values_from_csv_field('problem_tags', df)
        suggestion_tags = get_unique_values_from_csv_field('suggestion_tags', df)
        #display('...got em')        
    except:
        # these columns must not exist yet, so create them
        df['problem_tags'] = ''
        df['suggestion_tags'] = ''
        df['problem_tags'] = df['problem_tags'].astype(str)
        df['suggestion_tags'] = df['suggestion_tags'].astype(str)
        #display('...created em')
    
    clear_output() # wipe the output clear    
    
    display(Markdown('## Response #{}'.format(index)))
    
    # display each row's free text fields:
    for field in fields:
        # display this person's response to this question if they gave one
        value = row[field]
        if not pd.isnull(row[field]):
            display(Markdown('### {} comment\n{}'.format(field, value)))
        
    # ask the user for tags
    display(Markdown('## Tag this response'))
    display(Markdown("Enter tags that represent this person's complaints and suggestions.  Re-use tags used in others' responses, where applicable."))
    
    # display commonly used tags
    display(Markdown('### Common complaints (out of {} total)'.format(pd.Series(problem_tags).value_counts().sum())))
    display(pd.Series(problem_tags).value_counts())

    display(Markdown('### Common suggestions (out of {} total)'.format(pd.Series(suggestion_tags).value_counts().sum())))
    display(pd.Series(suggestion_tags).value_counts())
    
    # allow editor to tag this response
    my_problems = input("Complaints (or hit Enter to skip): ").strip()
    my_suggestions = input("Suggestions (or hit Enter to skip): ").strip()
    
    # store the tags in this record
    df.at[index,'problem_tags'] = my_problems
    df.at[index,'suggestion_tags'] = my_suggestions
    
    # update lists of tags that have already been used
    problem_tags = get_unique_values_from_csv_field('problem_tags', df)
    suggestion_tags = get_unique_values_from_csv_field('suggestion_tags', df)

    # save these tags to show later
    #p = [val.strip() for val in my_problems.split(',')]
    #s = [val.strip() for val in my_suggestions.split(',')]
    #problem_tags.extend(p)
    #suggestion_tags.extend(s)
    #problem_tags.sort()
    #suggestion_tags.sort()
    
    # increment counter
    counter += 1
    
    # save to file immediately just in case we have to interrupt the script mid-way
    df.to_csv('responses_scrubbed_tagged.csv', index=False)
    

## Save changes to a new CSV file

In [4]:
df.to_csv('responses_scrubbed_tagged.csv', index=False)